# 1. Labraries Loading

In [145]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

# 2. Dataset opening

In [146]:
measurementsDf = pd.read_json('../../../../Data/RawData/measurements.json')

In [147]:
measurementsDf.head(10)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,NaN,14.000000,NaN,NaN,@0123
1,2020-07-14T14:00:00.000Z,31.300000,NaN,44.475000,NaN,567.750000,D0001
2,2020-07-14T15:00:00.000Z,31.333333,NaN,45.505556,NaN,636.361111,D0001
3,2020-07-14T16:00:00.000Z,31.451429,NaN,47.145714,NaN,734.000000,D0001
4,2020-07-14T17:00:00.000Z,31.161765,NaN,36.000000,NaN,461.529412,D0001
5,2020-07-14T18:00:00.000Z,31.008333,NaN,28.436111,NaN,400.000000,D0001
6,2020-07-14T19:00:00.000Z,30.647222,NaN,29.133333,NaN,400.000000,D0001
7,2020-07-14T20:00:00.000Z,29.891429,NaN,28.597143,NaN,400.000000,D0001
8,2020-07-14T21:00:00.000Z,29.118919,NaN,29.162162,NaN,400.000000,D0001
9,2020-07-14T22:00:00.000Z,28.200000,NaN,31.611111,NaN,400.000000,D0001


In [133]:
print(measurementsDf.shape)

(117274, 7)


# 3. Data Preprocessing

In [134]:
# Its important to check for the null values

measurementsDf.isnull().sum()

time             0
T            38122
Rn            3160
H            38106
P            30649
CO2           3851
sensor_id        0
dtype: int64

In [122]:
# Null values visualization
#measurementsDf = measurementsDf.drop(['time'],axis = 1)
measurementsDf[measurementsDf['T'].isnull()]

,time,T,Rn,H,P,CO2,sensor_id


## 3.1 Data inputation using mean, median and most frequent from sklearn input librabry

### Mean inputation

In [71]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,648.532105,648.532105,648.532105,648.532105,@0123
1,2020-07-14T14:00:00.000Z,31.300000,648.532105,648.532105,648.532105,648.532105,D0001
2,2020-07-14T15:00:00.000Z,31.333333,648.532105,648.532105,648.532105,648.532105,D0001
3,2020-07-14T16:00:00.000Z,31.451429,648.532105,648.532105,648.532105,648.532105,D0001
4,2020-07-14T17:00:00.000Z,31.161765,648.532105,648.532105,648.532105,648.532105,D0001
5,2020-07-14T18:00:00.000Z,31.008333,648.532105,648.532105,648.532105,648.532105,D0001
6,2020-07-14T19:00:00.000Z,30.647222,648.532105,648.532105,648.532105,648.532105,D0001
7,2020-07-14T20:00:00.000Z,29.891429,648.532105,648.532105,648.532105,648.532105,D0001
8,2020-07-14T21:00:00.000Z,29.118919,648.532105,648.532105,648.532105,648.532105,D0001
9,2020-07-14T22:00:00.000Z,28.200000,648.532105,648.532105,648.532105,648.532105,D0001


### Median inputation

In [85]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,648.532105,648.532105,648.532105,648.532105,@0123
1,2020-07-14T14:00:00.000Z,31.300000,648.532105,648.532105,648.532105,648.532105,D0001
2,2020-07-14T15:00:00.000Z,31.333333,648.532105,648.532105,648.532105,648.532105,D0001
3,2020-07-14T16:00:00.000Z,31.451429,648.532105,648.532105,648.532105,648.532105,D0001
4,2020-07-14T17:00:00.000Z,31.161765,648.532105,648.532105,648.532105,648.532105,D0001
5,2020-07-14T18:00:00.000Z,31.008333,648.532105,648.532105,648.532105,648.532105,D0001
6,2020-07-14T19:00:00.000Z,30.647222,648.532105,648.532105,648.532105,648.532105,D0001
7,2020-07-14T20:00:00.000Z,29.891429,648.532105,648.532105,648.532105,648.532105,D0001
8,2020-07-14T21:00:00.000Z,29.118919,648.532105,648.532105,648.532105,648.532105,D0001
9,2020-07-14T22:00:00.000Z,28.200000,648.532105,648.532105,648.532105,648.532105,D0001


### Iteratve imputation

In [128]:
iterative_imputater = IterativeImputer()

In [129]:
measurementsDf['T'] =  iterative_imputater.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  iterative_imputater.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  iterative_imputater.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  iterative_imputater.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  iterative_imputater.fit_transform(measurementsDf[["Rn"]]).ravel()


measurementsDf.head(10)
#iterative_imputater.fit_transform(measurementsDf)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,648.532105,648.532105,648.532105,648.532105,@0123
1,2020-07-14T14:00:00.000Z,31.300000,648.532105,648.532105,648.532105,648.532105,D0001
2,2020-07-14T15:00:00.000Z,31.333333,648.532105,648.532105,648.532105,648.532105,D0001
3,2020-07-14T16:00:00.000Z,31.451429,648.532105,648.532105,648.532105,648.532105,D0001
4,2020-07-14T17:00:00.000Z,31.161765,648.532105,648.532105,648.532105,648.532105,D0001
5,2020-07-14T18:00:00.000Z,31.008333,648.532105,648.532105,648.532105,648.532105,D0001
6,2020-07-14T19:00:00.000Z,30.647222,648.532105,648.532105,648.532105,648.532105,D0001
7,2020-07-14T20:00:00.000Z,29.891429,648.532105,648.532105,648.532105,648.532105,D0001
8,2020-07-14T21:00:00.000Z,29.118919,648.532105,648.532105,648.532105,648.532105,D0001
9,2020-07-14T22:00:00.000Z,28.200000,648.532105,648.532105,648.532105,648.532105,D0001


### KNN imputation

In [148]:
knn_imputer = KNNImputer(n_neighbors = 4)

In [149]:
measurementsDf = measurementsDf.drop(['time','sensor_id'],axis = 1)
knn_imputer.fit_transform(measurementsDf)
measurementsDf.head(10)


,T,Rn,H,P,CO2
0,3.000000,NaN,14.000000,NaN,NaN
1,31.300000,NaN,44.475000,NaN,567.750000
2,31.333333,NaN,45.505556,NaN,636.361111
3,31.451429,NaN,47.145714,NaN,734.000000
4,31.161765,NaN,36.000000,NaN,461.529412
5,31.008333,NaN,28.436111,NaN,400.000000
6,30.647222,NaN,29.133333,NaN,400.000000
7,29.891429,NaN,28.597143,NaN,400.000000
8,29.118919,NaN,29.162162,NaN,400.000000
9,28.200000,NaN,31.611111,NaN,400.000000


In [144]:
measurementsDf.isnull().sum()


T      38122
Rn      3160
H      38106
P      30649
CO2     3851
dtype: int64

## 3.2 Data inputation using datawig library

In [ ]:
knn_imputer = KNNImputer()